### OpenAI Tutorial 7 Automatic Code Explainer
* This tutorial teaches you how to pass in a function to 

In [1]:
import os
import openai

# Import and pass api key
with open(r'C:\Users\mhuh22\Desktop\openai_api.txt', 'r') as file:
    api_key = file.readline().strip()
os.environ['OPENAI_API_KEY'] = api_key
openai.api_key = os.getenv('OPENAI_API_KEY')

In [2]:
# Python
# Function (OOP)
# """"""


In [3]:
def hello(name):
    print(f"Hello {name}")

In [4]:
hello('Matt')

Hello Matt


In [7]:
string = "is some words \"\"\" "

In [8]:
print(string)

is some words """ 


In [9]:
def docstring_prompt(code):
    prompt = f"{code}\n # A high quality python docstring of the above python function:\n\"\"\""
    return prompt

In [12]:
import inspect # Can be used to grab the entirety of a function

print(inspect.getsource(hello))

def hello(name):
    print(f"Hello {name}")



In [14]:
print(docstring_prompt(inspect.getsource(hello)))

def hello(name):
    print(f"Hello {name}")

 # A high quality python docstring of the above Python function:
 """


In [17]:
response = openai.Completion.create(
    model = 'text-davinci-003',
    prompt = docstring_prompt(inspect.getsource(hello)),
    temperature = 0,
    max_tokens = 100,
    top_p = 1.0,
    stop = ["\"\"\""]
)

In [18]:
print(response['choices'][0]['text'])


This function prints a greeting to the given name.

Parameters:
name (str): The name of the person to greet.

Returns:
None
 


In [19]:
# first line of the function (def myfunc():)
# """
# DOCSTRING (Completion API)
# """
# Rest of the function

<class 'str'>


In [81]:
def merge_docstring_and_function(original_function, docstring):
    function_string = inspect.getsource(original_function)
    split = function_string.split("\n")
    first_part, second_part = split[0], split[1:]
    docstring = docstring.replace('\n', '\n\t') # Inserts a tab after every new line
    merged_function = first_part + "\n" + '\t"""' + docstring + '"""\n' + "\n".join(second_part)
    return merged_function

In [82]:
print(merge_docstring_and_function(hello, response['choices'][0]['text']))

def hello(name):
	"""
	This function takes in a test string and the number of questions in the test as parameters and returns a dictionary with the questions as keys and the corresponding question text as values.
	
	Parameters:
	test (str): The test string containing the questions and answers.
	num_questions (int): The number of questions in the test.
	
	Returns:
	student_view (dict): A dictionary with the questions as keys and the corresponding question text as values.
	
	"""
    print(f"Hello {name}")



In [28]:
# def hello_v2 (name, pet):
#     print(f"Hello {name}")
#     print(f"feed {pet}")
#     print("all done")
    
#     second_part = inspect.getsource(hello_v2).split('\n')[1:]
# print("\n".join(second_part))

In [53]:
def test_function(test, num_questions):
    student_view = {1 : ""}
    question_number = 1
    for line in test.split("\n"):
        if not line.startswith("Correct Answer:"):
            student_view[question_number] += line+"\n"
        else:

            if question_number < num_questions:
                question_number+=1
                student_view[question_number] = ""
    return student_view

In [63]:
def create_docstring(function):
    response = openai.Completion.create(
      model= 'text-davinci-003',
      prompt=docstring_prompt(inspect.getsource(function)),
      temperature=0,
      max_tokens=256,
      top_p=1.0,
      frequency_penalty=0.0,
      presence_penalty=0.0,
      stop=["\"\"\""] # Corresponds to """, the end of the docstring
    )
    return merge_docstring_and_function(function, response['choices'][0]['text'])

In [66]:
print(create_docstring(test_function))

def test_function(test, num_questions):
    """   
This function takes in a test string and the number of questions in the test as parameters and returns a dictionary with the questions as keys and the corresponding question text as values.

Parameters:
test (str): The test string containing the questions and answers.
num_questions (int): The number of questions in the test.

Returns:
student_view (dict): A dictionary with the questions as keys and the corresponding question text as values.

    """
    student_view = {1 : ""}
    question_number = 1
    for line in test.split("\n"):
        if not line.startswith("Correct Answer:"):
            student_view[question_number] += line+"\n"
        else:

            if question_number < num_questions:
                question_number+=1
                student_view[question_number] = ""
    return student_view

